In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

Here We are implementing <b>KNN</b> based on some similarity measure<br>
There are many <b>similary measures</b> which are used to calculate the similarity between two data points.<br>
- <b>Similarity Measures between two pints $X_i$ and $X_j$ are</b>
    * Euclidian Distance $= \sqrt(X_i - X_j)^2$
    * Cosine similarity $= (X_i * X_j)\big/(||X_i||*||X_j||)$
    * so on

In [126]:
def euclianDistance(X,Q,Y):
    '''calculate Euclian distance'''
    X = tf.subtract(X,Q)
    X = tf.multiply(X,X)
    X = tf.reduce_sum(X,axis=1,keepdims=True)
    dis = tf.sqrt(X)
    Z = tf.concat((dis,Y),axis=1)
    return Z

def cosineDistance(X,Q,Y):
    p = tf.matmul(X,tf.transpose(Q))
    q = tf.linalg.norm(X,keepdims=True)*tf.linalg.norm(Q,keepdims=True)
    dis = 1.0 - tf.divide(p,q)
    Z = tf.concat((dis,Y),axis=1)
    return Z

In [127]:
from sklearn.datasets import load_iris
data = load_iris()
df = pd.DataFrame(data=data.data,columns=data.feature_names)
df['terget'] = data.target
t = np.where(df['terget']==2,0,df['terget'],)
X = df.iloc[:-1,:-1]
X = (X - X.mean())/X.std()
Y = pd.DataFrame(data=t[:-1],columns=['label'])
Y = np.float64(Y)

In [128]:
X = tf.constant(X)
Y = tf.constant(Y,dtype='float64')

In [129]:
Q = tf.constant(df.iloc[-1,:-1],shape=(1,4))

In [130]:
def predict(X, Y, Q, distnace=euclianDistance,k=2):
    '''
    X = training data
    Y = training labels
    Q = query point
    distance = any distance measure function that takes input (X,Y,Q) and return
    a tensor dim nX2, as first column represents the distance
    from the quey to sample at i th instance and 2nd column represent labels 
    of the corresponding sample. 
    k = k value for knn(2)
    '''
    dis = distnace(X,Q,Y)
    dis = tf.gather(dis, tf.argsort(dis[:,0]))
    return tf.reduce_max(dis[:k,1],keepdims=True)

In [135]:
dis = cosineDistance
Y_hat = predict(X,Y,Q,distnace=dis)

In [139]:
Y_hat == Y[-1]

<tf.Tensor: shape=(1,), dtype=bool, numpy=array([ True])>

# THE END